In [1]:
%pip install langchain langgraph openai faiss-cpu langchain-community langchain-openai grandalf tabulate


  Obtaining dependency information for tabulate from https://files.pythonhosted.org/packages/40/44/4a5f08c96eb108af5cb50b41f76142f0afa346dfa99d5296fe7202a11854/tabulate-0.9.0-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings,ChatOpenAI
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
import os



In [ ]:

#Load environment variables
load_dotenv(r'C:\Users\Rushil\Desktop\training\Agentic AI\LangGraph\env')
api_key = os.getenv("OPENAI_API_KEY")
hug_api_key = os.getenv("HUGGINGFACEHUB_API_KEY")

#Sample documents
docs = [
    Document(page_content="LangGraph is a library for building stateful, multi-agent applications with LangChain."),
    Document(page_content="LangChain is a framework for developing applications powered by language models.")
]

#  Initialize Hugging Face Embeddings (use any Sentence Transformers model)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#  Create FAISS vectorstore
vectorstore = FAISS.from_documents(docs, embedding_model)

#  Convert to retriever
retriever = vectorstore.as_retriever()


C:\Users\Rushil\AppData\Local\Temp\ipykernel_24632\3195515440.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [32]:
# Initialize LLM
llm = ChatOpenAI(temperature=0,model="gpt-4o-mini",api_key=api_key)

# Optional: memory if you want to extend with conversation history (not used in this minimal example)
memory = ConversationBufferMemory(return_messages=True)

# Rephrase question chain (optional)
rephrase_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that reformulates questions for better clarity."),
    ("human", "{input}")
])
rephrase_chain = LLMChain(llm=llm, prompt=rephrase_prompt)

# Answer generation chain
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Answer the question using the provided context."),
    ("human", "Context: {context}\n\nQuestion: {question}")
])
answer_chain = LLMChain(llm=llm, prompt=answer_prompt)


In [33]:
class QAState(dict):
    question: str = None 
    context: str = None
    answer: str = None

    
def input_node(state):
    return {"question": state["question"]}

def retrieve_node(state):
    docs = retriever.get_relevant_documents(state["question"])
    context = "\n".join([doc.page_content for doc in docs])
    print("\n\nContext retrieved from vectorstore:\n", context)
    return {"context": context}

def answer_node(state):
    answer = answer_chain.run(context=state["context"], question=state["question"])
    return {"answer": answer}

def output_node(state):
    print("\n SampleChatBot:", state["answer"])
    return state



In [34]:


builder = StateGraph(QAState)
builder.add_node("input", input_node)
builder.add_node("retrieve", retrieve_node)
builder.add_node("generate_answer", answer_node)
builder.add_node("output", output_node)

builder.set_entry_point("input")
builder.add_edge("input", "retrieve")
builder.add_edge("retrieve", "generate_answer")
builder.add_edge("generate_answer", "output")
builder.add_edge("output", END)

graph = builder.compile()


In [35]:
graph.get_graph().print_ascii()

   +-----------+     
   | __start__ |     
   +-----------+     
          *          
          *          
          *          
     +-------+       
     | input |       
     +-------+       
          *          
          *          
          *          
    +----------+     
    | retrieve |     
    +----------+     
          *          
          *          
          *          
+-----------------+  
| generate_answer |  
+-----------------+  
          *          
          *          
          *          
    +--------+       
    | output |       
    +--------+       
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      


In [36]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit","quit"]:
        print("Exiting chat. Goodbye!")
        break
    graph.invoke({"question": user_input})



Context retrieved from vectorstore:
 LangChain is a framework for developing applications powered by language models.
LangGraph is a library for building stateful, multi-agent applications with LangChain.


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}